In [1]:
import tensorflow as tf
import os
import pandas as pd
import math
import numpy as np

np.set_printoptions(suppress=True) # To display entire table
sess = tf.Session()

In [2]:
# Get the data file (should be in project root directory)
DATA_FILE = os.path.join(os.getcwd(), "heart.csv")
print(DATA_FILE)

C:\Users\Kildea\Desktop\tensorflowproject\heart.csv


In [3]:
# Read the data from the file
data = pd.read_csv(DATA_FILE)
data = data.sample(frac=1)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
70,54,1,2,120,258,0,0,147,0,0.4,1,0,3,1
85,67,0,2,115,564,0,0,160,0,1.6,1,0,3,1
176,60,1,0,117,230,1,1,160,1,1.4,2,2,3,0
64,58,1,2,140,211,1,0,165,0,0.0,2,0,2,1
219,48,1,0,130,256,1,0,150,1,0.0,2,2,3,0


In [4]:
# Normalize non-binary data and get column headers
headers = [] # For mapping indices to values
for col in data.columns:
    headers.append(col)
    if col in {"age", "cp", "trestbps", "chol", "thalach", "oldpeak", "slope", "ca", "thal"}:
        data[col]=(data[col]-data[col].min())/(data[col].max()-data[col].min())
        
data.head()
#print(headers)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
70,0.520833,1,0.666667,0.245283,0.301370,0,0,0.580153,0,0.064516,0.5,0.0,1.000000,1
85,0.791667,0,0.666667,0.198113,1.000000,0,0,0.679389,0,0.258065,0.5,0.0,1.000000,1
176,0.645833,1,0.000000,0.216981,0.237443,1,1,0.679389,1,0.225806,1.0,0.5,1.000000,0
64,0.604167,1,0.666667,0.433962,0.194064,1,0,0.717557,0,0.000000,1.0,0.0,0.666667,1
219,0.395833,1,0.000000,0.339623,0.296804,1,0,0.603053,1,0.000000,1.0,0.5,1.000000,0


In [5]:
data = data.to_numpy()

In [6]:
data_len = data.shape[0]
TRAINING_NUM_ROWS = math.ceil(0.7 * data_len)
VALIDATION_NUM_ROWS = math.floor(0.15 * data_len)
TESTING_NUM_ROWS = math.floor(0.15 * data_len)

print("{}   {}   {}".format(TRAINING_NUM_ROWS, VALIDATION_NUM_ROWS, TESTING_NUM_ROWS))

213   45   45


In [7]:
training_data_x = data[:TRAINING_NUM_ROWS, :13]
training_data_y = data[:TRAINING_NUM_ROWS, 13]
testing_data_x = data[TRAINING_NUM_ROWS:TESTING_NUM_ROWS+TRAINING_NUM_ROWS, :13]
testing_data_y = data[TRAINING_NUM_ROWS:TESTING_NUM_ROWS+TRAINING_NUM_ROWS, 13].astype(int)
validation_data_x = data[TESTING_NUM_ROWS+TRAINING_NUM_ROWS:, :13]
validation_data_y = data[TESTING_NUM_ROWS+TRAINING_NUM_ROWS:, 13]

validation_data_y = tf.keras.utils.to_categorical(validation_data_y)

In [8]:
# Prepare one-hot tensors
y_indices = [0, 1]
x_train_tensor = tf.convert_to_tensor(training_data_x)
y_train_tensor = tf.one_hot(training_data_y, 2)

In [9]:
def get_correct_percentage(results_raw):
    results_final = []

    for item in results_raw:
         results_final.append(np.where(item == max(item))[0][0])

    results_final = np.array(results_final)

    correct = 0

    for i in range(0, results_final.size):
        correct += 1 if (results_final[i] == testing_data_y[i]) else 0

    return 100 * correct/results_final.size

In [10]:
%%capture
# Sigmoid Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 1 hidden layer
model = tf.keras.Sequential()
hidden_layer = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
output_layer = tf.keras.layers.Dense(2, activation="softmax")
model.add(hidden_layer)
model.add(output_layer)

learning_rate = 0.2
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

model.compile(optimizer, loss=tf.keras.losses.categorical_crossentropy)
model.fit(x_train_tensor, y_train_tensor, batch_size=64, epochs=10, steps_per_epoch=100, validation_data=(validation_data_x, validation_data_y))

results1 = model.predict(testing_data_x);

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [11]:
%%capture
# Rectified Linear Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 1 hidden layer
model = tf.keras.Sequential()
hidden_layer = tf.keras.layers.Dense(16, input_dim=13, activation="relu")
output_layer = tf.keras.layers.Dense(2, activation="softmax")
model.add(hidden_layer)
model.add(output_layer)

learning_rate = 0.2
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

model.compile(optimizer, loss=tf.keras.losses.categorical_crossentropy)
model.fit(x_train_tensor, y_train_tensor, batch_size=64, epochs=10, steps_per_epoch=100, validation_data=(validation_data_x, validation_data_y))

results2 = model.predict(testing_data_x);

In [12]:
%%capture
# Sigmoid Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 3 hidden layers
model = tf.keras.Sequential()
hidden_layer_1 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
hidden_layer_2 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
hidden_layer_3 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
output_layer = tf.keras.layers.Dense(2, activation="softmax")
model.add(hidden_layer_1)
model.add(hidden_layer_2)
model.add(hidden_layer_3)
model.add(output_layer)

learning_rate = 0.2
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

model.compile(optimizer, loss=tf.keras.losses.categorical_crossentropy)
model.fit(x_train_tensor, y_train_tensor, batch_size=64, epochs=10, steps_per_epoch=100, validation_data=(validation_data_x, validation_data_y))

results3 = model.predict(testing_data_x);

In [13]:
%%capture
# Sigmoid Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 3 large hidden layers
model = tf.keras.Sequential()
hidden_layer_1 = tf.keras.layers.Dense(48, input_dim=13, activation="sigmoid")
hidden_layer_2 = tf.keras.layers.Dense(48, input_dim=13, activation="sigmoid")
hidden_layer_3 = tf.keras.layers.Dense(48, input_dim=13, activation="sigmoid")
output_layer = tf.keras.layers.Dense(2, activation="softmax")
model.add(hidden_layer_1)
model.add(hidden_layer_2)
model.add(hidden_layer_3)
model.add(output_layer)

learning_rate = 0.2
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

model.compile(optimizer, loss=tf.keras.losses.categorical_crossentropy)
model.fit(x_train_tensor, y_train_tensor, batch_size=64, epochs=10, steps_per_epoch=100, validation_data=(validation_data_x, validation_data_y))

results4 = model.predict(testing_data_x);

In [14]:
%%capture
# Sigmoid Activation Function with Adam Optimizer (Learning Rate 0.2), 1 hidden layer
model = tf.keras.Sequential()
hidden_layer_1 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
output_layer = tf.keras.layers.Dense(2, activation="softmax")
model.add(hidden_layer_1)
model.add(output_layer)

learning_rate = 0.2
optimizer = tf.train.AdamOptimizer(learning_rate)

model.compile(optimizer, loss=tf.keras.losses.categorical_crossentropy)
model.fit(x_train_tensor, y_train_tensor, batch_size=64, epochs=10, steps_per_epoch=100, validation_data=(validation_data_x, validation_data_y))

results5 = model.predict(testing_data_x);

In [15]:
%%capture
# Sigmoid Activation Function with Adam Optimizer (Learning Rate 0.2), 3 hidden layers
model = tf.keras.Sequential()
hidden_layer_1 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
hidden_layer_2 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
hidden_layer_3 = tf.keras.layers.Dense(16, input_dim=13, activation="sigmoid")
output_layer = tf.keras.layers.Dense(2, activation="softmax")
model.add(hidden_layer_1)
model.add(hidden_layer_2)
model.add(hidden_layer_3)
model.add(output_layer)

learning_rate = 0.2
optimizer = tf.train.AdamOptimizer(learning_rate)

model.compile(optimizer, loss=tf.keras.losses.categorical_crossentropy)
model.fit(x_train_tensor, y_train_tensor, batch_size=64, epochs=10, steps_per_epoch=100, validation_data=(validation_data_x, validation_data_y))

results6 = model.predict(testing_data_x);

In [16]:
'''
Method 1: Sigmoid Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 1 hidden layer
Method 2: Rectified Linear Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 1 hidden layer
Method 3: Sigmoid Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 3 hidden layers
Method 4: Sigmoid Activation Function with Gradient Descent Optimizer (Learning Rate 0.2), 3 large hidden layers
Method 5: Sigmoid Activation Function with Adam Optimizer (Learning Rate 0.2), 1 hidden layer
Method 6: Sigmoid Activation Function with Adam Optimizer (Learning Rate 0.2), 3 hidden layers
'''
result_percentages = []
result_percentages.append(get_correct_percentage(results1))
result_percentages.append(get_correct_percentage(results2))
result_percentages.append(get_correct_percentage(results3))
result_percentages.append(get_correct_percentage(results4))
result_percentages.append(get_correct_percentage(results5))
result_percentages.append(get_correct_percentage(results6))

for i in range(0, len(result_percentages)):
    print("Results for Method {}: {:.2f}%".format(i+1, result_percentages[i]))
    

    
print("\nBest result: Method {} @ {:.2f}%".format(result_percentages.index(max(result_percentages))+1, max(result_percentages)))

Results for Method 1: 88.89%
Results for Method 2: 86.67%
Results for Method 3: 91.11%
Results for Method 4: 91.11%
Results for Method 5: 80.00%
Results for Method 6: 82.22%

Best result: Method 3 @ 91.11%
